<a href="https://colab.research.google.com/github/mukkatharun/advance-deep-learning-assignments/blob/main/assignment2_GraphNeuralNetworks/link_prediction_of_graph_using_Graph_AutoEncoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing required packages

In [16]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

1.12.1+cu113


Loading the libraries

In [17]:
import torch
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from torch_geometric.utils import train_test_split_edges
from torch_geometric.nn import GAE

Loading CiteSeer dataset

In [18]:
dataset = Planetoid("\..", "CiteSeer", transform=T.NormalizeFeatures())
dataset.data

Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])

In [19]:
data = dataset[0]
data.train_mask = data.val_mask = data.test_mask = None
data

Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327])

In [20]:
data = train_test_split_edges(data)

/usr/local/lib/python3.7/dist-packages/torch_geometric/deprecation.py:12: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


Encoder Initialization code

In [21]:
class GCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True) # cached only for transductive learning
        self.conv2 = GCNConv(2 * out_channels, out_channels, cached=True) # cached only for transductive learning

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)
    

Initialization of Auto Encoder

In [22]:
out_channels = 2
num_features = dataset.num_features
epochs = 100

model = GAE(GCNEncoder(num_features, out_channels))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [23]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    #if args.variational:
    #   loss = loss + (1 / data.num_nodes) * model.kl_loss()
    loss.backward()
    optimizer.step()
    return float(loss)


In [24]:
def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)


In [25]:
for epoch in range(1, epochs + 1):
    loss = train()

    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    if epoch % 10 == 0:
      print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))


Epoch: 010, AUC: 0.6333, AP: 0.7034
Epoch: 020, AUC: 0.6378, AP: 0.7225
Epoch: 030, AUC: 0.6633, AP: 0.7355
Epoch: 040, AUC: 0.7563, AP: 0.7795
Epoch: 050, AUC: 0.7847, AP: 0.7987
Epoch: 060, AUC: 0.7907, AP: 0.8039
Epoch: 070, AUC: 0.7932, AP: 0.8051
Epoch: 080, AUC: 0.7951, AP: 0.8060
Epoch: 090, AUC: 0.7930, AP: 0.8037
Epoch: 100, AUC: 0.7918, AP: 0.8024


In [26]:
Z = model.encode(x, train_pos_edge_index)
Z

tensor([[ 0.2787, -0.2904],
        [-0.9910,  0.9829],
        [ 0.7703, -0.7630],
        ...,
        [-0.3345,  0.2937],
        [ 0.5873, -0.6182],
        [ 0.8147, -0.8141]], grad_fn=<AddBackward0>)